In [199]:
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen as urq
import requests
import pandas as pd
import numpy as np

### Scrapping phones from Amazon website

In [200]:
url='https://www.amazon.in/s/ref=mega_elec_s23_1_1_1_1?rh=i%3Aelectronics%2Cn%3A1389401031&ie=UTF8&bbn=976419031'

In [201]:
client=urq(url)

In [202]:
page=client.read()

In [203]:
client.close()

In [204]:
page_soup=bs(page,'html.parser')

In [205]:
container=page_soup.find_all('div',{'class':'s-item-container'})
#prices=page_soup.find_all('span',{'class':'a-size-base a-color-price s-price a-text-bold'})

In [206]:
print(len(container))

25


### Scrapping Model, Price and RAM

In [207]:
price_inr=[];model=[];ram=[]
for obj in container[:-1]:
    name=obj.find('img')['alt']
    model.append(name)
    s=obj.find('span',{'class':'a-size-base a-color-price s-price a-text-bold'})
    if s==None:
        price_inr.append(None)
    else:
        price=(bs.prettify(s).split('\n'))
        price_inr.append(int(price[3][1:].replace(',','')))
    if 'RAM,' in name.split(' '):
        i=name.split(' ').index('RAM,')
        ram.append(int(name.split(' ')[i-1][0]))
    else:
        ram.append(None)

In [208]:
print(price_inr,len(price_inr),model,len(model),ram,sep='\n\n')

[11499, 1999, 11499, None, 11499, 7299, 7999, 16999, 249, 12699, 599, 229, 16999, 1499, 1999, 799, 14999, 15999, 7999, 1499, 269, 3999, 399, 399]

24

['Redmi Note 8 (Neptune Blue, 4GB RAM, 64GB Storage)', 'OnePlus Bullets Wireless Z (Black)', 'Redmi Note 8 (Moonlight White, 4GB RAM, 64GB Storage)', 'Redmi 8A Dual (Sea Blue, 2GB RAM, 32GB Storage) – Dual Cameras & 5,000 mAH Battery', 'Redmi Note 8 (Space Black, 4GB RAM, 64GB Storage)', 'Redmi 8A Dual (Sky White, 2GB RAM, 32GB Storage) – Dual Cameras & 5,000 mAH Battery', 'Redmi 8A Dual (Sea Blue, 3GB RAM, 32GB Storage) – Dual Cameras & 5,000 mAH Battery', 'Samsung Galaxy M31 (Ocean Blue, 6GB RAM, 128GB Storage)', 'boAt Rugged v3 Extra Tough Unbreakable Braided Micro USB Cable 1.5 Meter (Black)', 'Samsung Galaxy M21 (Raven Black, 4GB RAM, 64GB Storage)', 'JBL C100SI In-Ear Deep Bass Headphones with Mic (Black)', 'Mi USB Cable 120cm (Black)', 'Samsung Galaxy M31 (Space Black, 6GB RAM, 128GB Storage)', 'boAt Rockerz 255 Sports Wireless He

### Scrapping other pages

In [209]:
urls=[]
urls.append('https://www.amazon.in/s?rh=n%3A976419031%2Cn%3A%21976420031%2Cn%3A1389401031&page=2&qid=1589360014&ref=lp_1389401031_pg_2')
urls.append('https://www.amazon.in/s?i=electronics&rh=n%3A976419031%2Cn%3A976420031%2Cn%3A1389401031&page=3&qid=1589361210&ref=sr_pg_3')
urls.append('https://www.amazon.in/s?i=electronics&rh=n%3A976419031%2Cn%3A976420031%2Cn%3A1389401031&page=4&qid=1589361223&ref=sr_pg_4')
urls.append('https://www.amazon.in/s?i=electronics&rh=n%3A976419031%2Cn%3A976420031%2Cn%3A1389401031&page=6&qid=1589362577&ref=sr_pg_6')
for url in urls:
    client=urq(url)
    page=client.read()
    client.close()
    page_soup=bs(page,'html.parser')
    container=page_soup.find_all('div',{'class':'a-section a-spacing-medium'})
    for obj in container:
        name=obj.find('img')['alt']
        model.append(name)
        s=obj.find('span',{'class':'a-price-whole'})
        if s==None:
            price_inr.append(None)
        else:
            price=(bs.prettify(s).split('\n'))
            #print(price,int(price[1][1:].replace(',','')))
            price_inr.append(int(price[1][1:].replace(',','')))
        if 'RAM,' in name.split(' '):
            i=name.split(' ').index('RAM,')
            ram.append(int(name.split(' ')[i-1][0]))
        else:
            ram.append(None)

### creating a dataframe and exporting data to a csv file

In [210]:
print(len(model),len(price_inr),len(ram))

120 120 120


In [211]:
data=pd.DataFrame({'Product':model,'Price':price_inr,'RAM':ram})

### Drop all products that are not phones

In [212]:
data[data['RAM'].isnull()].head()

,Product,Price,RAM
1,OnePlus Bullets Wireless Z (Black),1999.0,NaN
8,boAt Rugged v3 Extra Tough Unbreakable Braided...,249.0,NaN
10,JBL C100SI In-Ear Deep Bass Headphones with Mi...,599.0,NaN
11,Mi USB Cable 120cm (Black),229.0,NaN
13,boAt Rockerz 255 Sports Wireless Headset with ...,1499.0,NaN


In [213]:
data.dropna(axis=0,inplace=True)

In [214]:
data.head()

,Product,Price,RAM
0,"Redmi Note 8 (Neptune Blue, 4GB RAM, 64GB Stor...",11499.0,4.0
2,"Redmi Note 8 (Moonlight White, 4GB RAM, 64GB S...",11499.0,4.0
4,"Redmi Note 8 (Space Black, 4GB RAM, 64GB Storage)",11499.0,4.0
5,"Redmi 8A Dual (Sky White, 2GB RAM, 32GB Storag...",7299.0,2.0
6,"Redmi 8A Dual (Sea Blue, 3GB RAM, 32GB Storage...",7999.0,3.0


In [215]:
data.to_excel('phones.xlsx',index=False)

In [216]:
data.describe()

,Price,RAM
count,35.000000,35.000000
mean,16583.685714,5.200000
std,9294.396344,1.659199
min,7299.000000,2.000000
25%,11499.000000,4.000000
50%,14999.000000,6.000000
75%,16999.000000,6.000000
max,47999.000000,8.000000


In [217]:
max_RAM=data['RAM'].max()

In [218]:
new_data=data[data['RAM']==max_RAM].sort_values(by='Price',axis=0)

In [219]:
new_data

,Product,Price,RAM
100,"Poco F1 by Xiaomi (Armored Edition, 8GB RAM, 2...",16999.0,8.0
27,"OnePlus 7T (Glacier Blue, 8GB RAM, Fluid AMOLE...",34999.0,8.0
57,"OnePlus 7T (Frosted Silver, 8GB RAM, Fluid AMO...",34999.0,8.0
77,"OnePlus 7 Pro (Mirror Grey, 8GB RAM, Fluid AMO...",42999.0,8.0
99,"OnePlus 7T Pro (Haze Blue, 8GB RAM, Fluid AMOL...",47999.0,8.0


## The phone with highest RAM at lowest price

In [220]:
new_data.head(1)

,Product,Price,RAM
100,"Poco F1 by Xiaomi (Armored Edition, 8GB RAM, 2...",16999.0,8.0
